In [0]:
file = dbutils.widgets.text("file", "")
file = dbutils.widgets.get("file")

In [0]:
# Creamos catalogo y schema
catalog_name = "sesion_08"
schema_bronze = "bronze"
schema_silver = "silver"
schema_gold   = "gold"

In [0]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_bronze}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_silver}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_gold}")

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.default.source")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.{schema_bronze}.ventas(
  id_venta string,
  id_linea string,
  id_tienda string,
  id_cliente string,
  id_producto string,
  fecha_venta string,
  cantidad string,
  precio_unitario string,
  updated_at string,
  ingest_at timestamp
)
"""
)

In [0]:
from pyspark.sql.functions import col, to_date, to_timestamp, row_number, current_timestamp, current_date, sha2, concat_ws, lit, expr
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from delta.tables import DeltaTable
import uuid

In [0]:
["ventas_base","ventas_incremento"]

In [0]:
# Definimos rutas de origen
path_base = "/Volumes/dmc_01/default/volumen_01/sesion_08/default/source/" 

path_ventas  = f"{path_base}/{file}.csv"

In [0]:
# Lectura
ventas = (
    spark.read.option("header", True).csv(path_ventas)
    .withColumn("ingest_at", current_timestamp())
)

In [0]:
ventas.createOrReplaceTempView("ventas_bronze")

spark.sql(f"""
INSERT INTO {catalog_name}.{schema_bronze}.ventas
SELECT * FROM ventas_bronze          
""")